## Week 3 Final Challenge

In [108]:
import numpy as np
from qiskit import IBMQ, QuantumCircuit, Aer, execute, QuantumRegister, ClassicalRegister
from qiskit.tools.jupyter import *
from qiskit.quantum_info import Operator
from qiskit.visualization import plot_histogram
%matplotlib inline
from qiskit import QuantumCircuit
from qiskit.compiler import transpile
from qiskit.transpiler import PassManager
import matplotlib.pyplot as plt
provider = IBMQ.load_account()
backend = provider.get_backend('ibmq_qasm_simulator')

ibmqfactory.load_account:WARNING:2020-11-25 13:06:48,610: Credentials are already in use. The existing account in the session will be replaced.


In [2]:
problem_set = \
    [[['0', '2'], ['1', '0'], ['1', '2'], ['1', '3'], ['2', '0'], ['3', '3']],
    [['0', '0'], ['0', '1'], ['1', '2'], ['2', '2'], ['3', '0'], ['3', '3']],
    [['0', '0'], ['1', '1'], ['1', '3'], ['2', '0'], ['3', '2'], ['3', '3']],
    [['0', '0'], ['0', '1'], ['1', '1'], ['1', '3'], ['3', '2'], ['3', '3']],
    [['0', '2'], ['1', '0'], ['1', '3'], ['2', '0'], ['3', '2'], ['3', '3']],
    [['1', '1'], ['1', '2'], ['2', '0'], ['2', '1'], ['3', '1'], ['3', '3']],
    [['0', '2'], ['0', '3'], ['1', '2'], ['2', '0'], ['2', '1'], ['3', '3']],
    [['0', '0'], ['0', '3'], ['1', '2'], ['2', '2'], ['2', '3'], ['3', '0']],
    [['0', '3'], ['1', '1'], ['1', '2'], ['2', '0'], ['2', '1'], ['3', '3']],
    [['0', '0'], ['0', '1'], ['1', '3'], ['2', '1'], ['2', '3'], ['3', '0']],
    [['0', '1'], ['0', '3'], ['1', '2'], ['1', '3'], ['2', '0'], ['3', '2']],
    [['0', '0'], ['1', '3'], ['2', '0'], ['2', '1'], ['2', '3'], ['3', '1']],
    [['0', '1'], ['0', '2'], ['1', '0'], ['1', '2'], ['2', '2'], ['2', '3']],
    [['0', '3'], ['1', '0'], ['1', '3'], ['2', '1'], ['2', '2'], ['3', '0']],
    [['0', '2'], ['0', '3'], ['1', '2'], ['2', '3'], ['3', '0'], ['3', '1']],
    [['0', '1'], ['1', '0'], ['1', '2'], ['2', '2'], ['3', '0'], ['3', '1']]]

In [59]:
# Creates List for Permutation Matrices

l=[]
for i in range(4):
    list4 = [0,1,2,3]
    list3 = list(list4)
    list3.remove(i)
    for j in list3:
        list2 = list(list3)
        list2.remove(j)
        for k in list2:
            list1 = list(list2)
            list1.remove(k)
            l.append([i,(j+4),(k+8),(list1[0]+12)])

print(l)

[[0, 5, 10, 15], [0, 5, 11, 14], [0, 6, 9, 15], [0, 6, 11, 13], [0, 7, 9, 14], [0, 7, 10, 13], [1, 4, 10, 15], [1, 4, 11, 14], [1, 6, 8, 15], [1, 6, 11, 12], [1, 7, 8, 14], [1, 7, 10, 12], [2, 4, 9, 15], [2, 4, 11, 13], [2, 5, 8, 15], [2, 5, 11, 12], [2, 7, 8, 13], [2, 7, 9, 12], [3, 4, 9, 14], [3, 4, 10, 13], [3, 5, 8, 14], [3, 5, 10, 12], [3, 6, 8, 13], [3, 6, 9, 12]]


In [102]:
def qRam4FA(qc, DataQ, AdrQ, AncQ, problem_set):
    anc = [AncQ[3], AncQ[4], AncQ[5]]
    boards = []
    for i in range(len(problem_set)):
        zeros = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
        boards.append(zeros)
        for j in range(len(problem_set[i])):
            problem_set[i][j][0] = int(problem_set[i][j][0])
            problem_set[i][j][1] = int(problem_set[i][j][1])
            ro = problem_set[i][j][0]
            col = problem_set[i][j][1]
            index = ro*4 + col
            boards[i][index] = 1
    for adr in range(16):
        bin_adr = [int(x) for x in '{:04b}'.format(adr)]
        for i in range(4):
            if bin_adr[i] == 0:
                qc.x(AdrQ[i])
        for i in range(16):
            if boards[adr][i] == 1:
                qc.mct(control_qubits=AdrQ, target_qubit=DataQ[i], ancilla_qubits=anc, mode='basic')
        for i in range(4):
            if bin_adr[i] == 0:
                qc.x(AdrQ[i])

In [103]:
def Oracle4FA(qc, DataQ, FlagQ, AncQ):
    for i in range(len(l)):
        lsit = []
        for j in range(4):
            lsit.append(DataQ[(l[i][j])])  
        qc.rccx(lsit[0],lsit[1],AncQ[3])
        qc.rcccx(lsit[2],lsit[3],AncQ[3],AncQ[0])
        qc.rccx(lsit[0],lsit[1],AncQ[3])
        qc.cx(AncQ[0], AncQ[1])
        qc.x(AncQ[1])
        qc.rccx(AncQ[0], AncQ[1], AncQ[2])
        qc.x(AncQ[1])
        qc.rccx(lsit[0],lsit[1],AncQ[3])
        qc.rcccx(lsit[2],lsit[3],AncQ[3],AncQ[0])
        qc.rccx(lsit[0],lsit[1],AncQ[3])
        
    qc.cx(AncQ[1], FlagQ)
    qc.cx(AncQ[2], FlagQ)
    qc.rccx(AncQ[1], AncQ[2], FlagQ)

    for i in range(len(l)):
        lsit = []
        for j in range(4):
            lsit.append(DataQ[(l[(len(l)-i-1)][j])])  
        qc.rccx(lsit[0],lsit[1],AncQ[3])
        qc.rcccx(lsit[2],lsit[3],AncQ[3],AncQ[0])
        qc.rccx(lsit[0],lsit[1],AncQ[3])
        qc.x(AncQ[1])
        qc.rccx(AncQ[0], AncQ[1], AncQ[2])
        qc.x(AncQ[1])
        qc.cx(AncQ[0], AncQ[1])
        qc.rccx(lsit[0],lsit[1],AncQ[3])
        qc.rcccx(lsit[2],lsit[3],AncQ[3],AncQ[0])
        qc.rccx(lsit[0],lsit[1],AncQ[3])


In [104]:
def phase_oracle(n, indices_to_mark, name = 'Oracle'):
    qc = QuantumCircuit(n, name=name)
    oracle_matrix = np.identity(2**n)
    for i in indices_to_mark:
        oracle_matrix[i, i] = -1
    qc.unitary(Operator(oracle_matrix), range(n))
    return qc

In [105]:
def diffuser(n):
    qc = QuantumCircuit(n, name='Diff - "V"')
    qc.h(range(n))
    qc.append(phase_oracle(n,[0]), range(n))
    qc.h(range(n))
    return qc

In [107]:
AdrQ = QuantumRegister(4, name='adr')
DataQ = QuantumRegister(16, name='data')
FlagQ = QuantumRegister(1, name='flag')
AncQ = QuantumRegister(6, name='anc')
Out = ClassicalRegister(4, name='out')
qc = QuantumCircuit(DataQ, AdrQ, FlagQ, AncQ, Out)

qc.x(FlagQ)
qc.h(FlagQ)
qc.h(AdrQ)

qRam4FA(qc, DataQ, AdrQ, AncQ, problem_set)
Oracle4FA(qc, DataQ, FlagQ, AncQ)
qRam4FA(qc, DataQ, AdrQ, AncQ, problem_set)
qc.append(diffuser(4),AdrQ)

qc.measure(AdrQ, Out)
qc = qc.reverse_bits()
qc.compiler.transpile()
result = execute(qc, backend, shots = 8192).result()
counts = result.get_counts()
plot_histogram(counts)

AttributeError: 'QuantumCircuit' object has no attribute 'compiler'

In [9]:
qc.draw()

┌───┐                                                      ┌─┐
  adr_0: ┤ H ├──────────────────────────────────────────────────────┤M├
         ├───┤                                                   ┌─┐└╥┘
  adr_1: ┤ H ├───────────────────────────────────────────────────┤M├─╫─
         ├───┤                                                ┌─┐└╥┘ ║ 
  adr_2: ┤ H ├────────────────────────────────────────────────┤M├─╫──╫─
         ├───┤                                             ┌─┐└╥┘ ║  ║ 
  adr_3: ┤ H ├─────────────────────────────────────────────┤M├─╫──╫──╫─
         └───┘                                          ┌─┐└╥┘ ║  ║  ║ 
 data_0: ───────────────────────────────────────────────┤M├─╫──╫──╫──╫─
                                                     ┌─┐└╥┘ ║  ║  ║  ║ 
 data_1: ────────────────────────────────────────────┤M├─╫──╫──╫──╫──╫─
                                                  ┌─┐└╥┘ ║  ║  ║  ║  ║ 
 data_2: ─────────────────────────────────────────┤M├─╫──╫──╫──╫──╫──╫─
                                               ┌─┐└╥┘ ║  ║  ║  ║  ║  ║ 
 data_3: ──────────────────────────────────────┤M├─╫──╫──╫──╫──╫──╫──╫─
                                            ┌─┐└╥┘ ║  ║  ║  ║  ║  ║  ║ 
 data_4: ───────────────────────────────────┤M├─╫──╫──╫──╫──╫──╫──╫──╫─
                                         ┌─┐└╥┘ ║  ║  ║  ║  ║  ║  ║  ║ 
 data_5: ────────────────────────────────┤M├─╫──╫──╫──╫──╫──╫──╫──╫──╫─
                                      ┌─┐└╥┘ ║  ║  ║  ║  ║  ║  ║  ║  ║ 
 data_6: ─────────────────────────────┤M├─╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─
                                   ┌─┐└╥┘ ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ 
 data_7: ──────────────────────────┤M├─╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─
                                ┌─┐└╥┘ ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ 
 data_8: ───────────────────────┤M├─╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─
                             ┌─┐└╥┘ ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ 
 data_9: ────────────────────┤M├─╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─
                          ┌─┐└╥┘ ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ 
data_10: ─────────────────┤M├─╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─
                       ┌─┐└╥┘ ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ 
data_11: ──────────────┤M├─╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─
                    ┌─┐└╥┘ ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ 
data_12: ───────────┤M├─╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─
                 ┌─┐└╥┘ ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ 
data_13: ────────┤M├─╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─
              ┌─┐└╥┘ ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ 
data_14: ─────┤M├─╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─
          ┌─┐ └╥┘ ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ 
data_15: ─┤M├──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─
          └╥┘  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ 
 out: 20/══╩═══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩═
           15  14 13 12 11 10 9  8  7  6  5  4  3  2  1  0  19 18 17 16